<a href="https://colab.research.google.com/github/Nithinps021/Project_IISU/blob/main/Chatbot_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib import response
import numpy as np
import random
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Bidirectional, Dropout, Dense, Activation, Flatten, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import SGD

data_file = open('intents_cleaned_1.json').read()
intents = json.loads(data_file)
intents=intents['intents']
random.shuffle(intents)
classes=[]
alldata=[]
responses={}
for i in intents:
	if i['tag'] not in classes:
		classes.append(i['tag'])
		responses[i['tag']]=i['responses'][0]
	for j in i['patterns']:
		alldata.append([j,i['tag']])


random.shuffle(alldata)
tot_data=len(alldata)
tot_classes=len(classes)
x_train=np.array([])
y_train=np.zeros(shape=(tot_data,tot_classes), dtype=np.uint8)
for i in range(tot_data):
  x_train=np.append(x_train,alldata[i][0])
  y_train[i][classes.index(alldata[i][1])]=1

text=x_train
Tokenizer = Tokenizer()
Tokenizer.fit_on_texts(text) 
Tokenizer_vocab_size = len(Tokenizer.word_index) + 1
Tokenizer_vocab_size, x_train.shape, y_train.shape


x_val=x_train[0:157]
y_val=y_train[0:157]

x_train=x_train[157:]
y_train=y_train[157:]

X_train_encoded_words = Tokenizer.texts_to_sequences(x_train)
X_val_encoded_words = Tokenizer.texts_to_sequences(x_val)
X_train_encoded_padded_words = sequence.pad_sequences(X_train_encoded_words, maxlen =64)
X_val_encoded_padded_words = sequence.pad_sequences(X_val_encoded_words, maxlen = 64)



model = Sequential()

model.add(Embedding(Tokenizer_vocab_size, 32, input_length = 64)) # Accepts the vector inputs
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64))) 
model.add(Dropout(0.5))
model.add(Dense(800, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(tot_classes, activation='softmax'))

model.summary()

Nadam = tf.keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
model.compile(loss='categorical_crossentropy', optimizer=Nadam, metrics=['accuracy'])
history  = model.fit(X_train_encoded_padded_words,y_train, epochs = 100, batch_size=100, verbose=1, validation_data=(X_val_encoded_padded_words, y_val))
model.save('gru3.h5')

print(len(x_train),y_train)
print(Tokenizer_vocab_size)






Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 64, 32)            38752     
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              37632     
 nal)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 800)               103200    
                                                                 
 dropout_7 (Dropout)         (None, 800)               0         
                                                                 
 dense_7 (Dense)             (None, 200)               160200    
                                                      

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


14/14 [==============================] - 7s 180ms/step - loss: 5.7436 - accuracy: 0.0066 - val_loss: 5.7473 - val_accuracy: 0.0255
Epoch 2/100
14/14 [==============================] - 2s 118ms/step - loss: 5.7289 - accuracy: 0.0153 - val_loss: 5.7581 - val_accuracy: 0.0255
Epoch 3/100
14/14 [==============================] - 2s 115ms/step - loss: 5.6875 - accuracy: 0.0153 - val_loss: 5.6792 - val_accuracy: 0.0255
Epoch 4/100
14/14 [==============================] - 2s 117ms/step - loss: 5.5799 - accuracy: 0.0153 - val_loss: 5.5727 - val_accuracy: 0.0255
Epoch 5/100
14/14 [==============================] - 2s 118ms/step - loss: 5.4464 - accuracy: 0.0153 - val_loss: 5.4996 - val_accuracy: 0.0255
Epoch 6/100
14/14 [==============================] - 2s 122ms/step - loss: 5.2628 - accuracy: 0.0138 - val_loss: 5.3708 - val_accuracy: 0.0382
Epoch 7/100
14/14 [==============================] - 2s 130ms/step - loss: 5.0294 - accuracy: 0.0277 - val_loss: 5.1959 - val_accuracy: 0.0446
Epoch 8/100

In [ ]:
print('Chatbot started. type /stop to stop the bot')
def runChatbot():
	while(True):
		sen=input('Qurey : ')
		if sen=='/stop':
			break
		tokens = Tokenizer.texts_to_sequences([sen])
		tokens = pad_sequences(tokens, maxlen = 64)
		prediction = model.predict(np.array(tokens))
		pred = np.argmax(prediction)
		print('RESPONSE : ',responses[classes[pred]])
runChatbot()

Chatbot started. type /stop to stop the bot
RESPONSE :  The Indian Space Research Organization ISRO is India's national space agency. India has been steadily building its launching and exploration capabilities for decades
RESPONSE :   Geosynchronous Satellite Launch Vehicle Mark II GSLV Mk II is the largest launch vehicle developed by India, which is currently in operation
RESPONSE :  Polar Satellite Launch Vehicle
RESPONSE :     Robotic helicopter which could fly on Mars 
